# EXTRACAO DOS FRAMES DOS VIDEOS DE CADA ANDAR

Preencher informacoes:

In [6]:
# Diretorio dos videos
dir_vids = r"D:\VIDYA\modec2"

# Output com os frames que sera criado pelo codigo
dir_frames = r"D:\VIDYA" # raiz onde sera cirada a pasta output-frames

In [7]:
# Frames por segundo
fps = 2

Codigo:

In [8]:
import os
import subprocess
import pandas as pd

In [9]:
# Retorna os nomes dos diretorios avo, pai, atual e o caminho completo do atual
def get_info_modulo(dirpath):
    
    # Separar o caminho em partes
    parts = dirpath.split(os.sep)
    current_index = len(parts) - 1

    # Obter o diretorio atual, pai e avô
    current_dir = parts[current_index] if current_index >= 0 else ''
    parent_dir = parts[current_index - 1] if current_index - 1 >= 0 else ''
    grandparent_dir = parts[current_index - 2] if current_index - 2 >= 0 else ''

    # Utilizando o caminho global, corrigdo para ter '/' dps da unidade de disco, senao da erro
    full_path = os.path.join(*parts)
    if f':{os.sep}' not in full_path:
        full_path = full_path.replace(":", f':{os.sep}')
    
    return grandparent_dir, parent_dir, current_dir, full_path

In [10]:
# Extrai dos frames
def extrair_frames(root_dir, dir_output_frames):
    if ' ' in root_dir or ' ' in dir_output_frames:
        print("Erro: caminhos dos videos e dos frames nao devem conter espacos!")
    else:
        try:
            for dirpath, dirnames, filenames in os.walk(root_dir): # percorre os diretorios dos videos

                # Obtencao das infos do video
                ativo, modulo, pasta_andar, full_path = get_info_modulo(dirpath)
                andar = pasta_andar.replace(modulo, "").replace('_floor_', "")


                for filename in filenames: # pega cada arquivo
                    try:
                        if filename.lower().endswith(('.mp4', '.avi', '.mov', '.mkv', '.flv')): # pega os arquivos que forem videos
                            
                            # Local do video a ser processado
                            input_video = os.path.join(full_path, filename)

                            # Criacao e localizacao da pasta de output dos frames
                            output_dir = os.path.join(dir_output_frames, f'output-frames/{ativo}/{modulo}/{modulo}_floor_{andar}') # formato do output
                            os.makedirs(output_dir, exist_ok=True) # cria o output para os frames, se nao existir
                            output = os.path.join(output_dir, f"{filename.replace('.mp4', '')}_{ativo}_{modulo}_{andar}_%05d.jpg") # pega a localizacao de onde ficarao os frames do andar atual
                                                                                                                    # importante: %04 -> 0001, 0002, ... , 9999;  %05 -> 00001 ... 99999

                            print(f'\nProcessando video {input_video} - {modulo} - {andar}')
                            # Executa o comando do FFmpeg pelo subprocess para extrair os frames
                            comando = f'ffmpeg -i "{input_video}" -vf fps={fps} -qscale:v 1 "{output}"'
                            

                            resultado = subprocess.run(comando, shell=True, text=True, capture_output=True, encoding='utf-8')

                            # Printa o resultado do FFmpeg
                            print("\n\n", comando)
                            print("\nSaída:\n", resultado.stdout.strip())
                            print("Erros:\n", resultado.stderr.strip())
                    except KeyboardInterrupt:
                        print("Execucucao interrompida manualmente")  
                    except:
                        print("Erro: possivel arquivo corrompido")

        except KeyboardInterrupt:
            print("Execucucao interrompida manualmente")
        except:
            print("Erro: provavelmente nao suportou tantos arquivos")


In [11]:
# Extrai os Frames
extrair_frames(dir_vids, dir_frames)


Processando video D:\VIDYA\modec2\TOPSIDE\1P\1 ANDAR\VID_20180103_222728_00_001.mp4 - 1P - 1 ANDAR


 ffmpeg -i "D:\VIDYA\modec2\TOPSIDE\1P\1 ANDAR\VID_20180103_222728_00_001.mp4" -vf fps=2 -qscale:v 1 "D:\VIDYA\output-frames/TOPSIDE/1P/1P_floor_1 ANDAR\VID_20180103_222728_00_001_TOPSIDE_1P_1 ANDAR_%05d.jpg"

Saída:
 
Erros:
 ffmpeg version 7.0.2-essentials_build-www.gyan.dev Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 13.2.0 (Rev5, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-zlib --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-sdl2 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-libaom --enable-libopenjpeg --enable-libvpx --enable-mediafoundation --enable-libass --enable-libfreetype --enable-libfribidi --enab